In [10]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 3.1 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 3.0 MB/s eta 0:00:0000:0100:02


In [82]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [83]:
data = pd.read_csv( 'hackology_2024/axe_transactions.csv' )

In [84]:
data

,transaction_datetime,customer_id,location,product_name,category,size_cm,weight_kg,quantity,unit_price,discount,total_amount,payment_method,day_of_year,cost_price,profit,transaction_month,transaction_year
0,2018-01-01 17:34:00,67,3,Small Axe,Hand Tools,35,1.5,5,30.00,15,128,Cash,1,12,68,1,2018
1,2018-01-01 09:28:00,49,9,Battle Axe,Combat Tools,95,10.0,2,120.00,15,204,Mobile Payment,1,48,108,1,2018
2,2018-01-01 18:57:00,20,5,Battle Axe,Combat Tools,95,10.0,5,120.00,15,510,Mobile Payment,1,48,270,1,2018
3,2018-01-01 09:43:00,44,5,Large Axe,Outdoor Equipment,75,6.0,5,80.00,15,340,Credit Card,1,32,180,1,2018
4,2018-01-01 10:01:00,26,13,Large Axe,Outdoor Equipment,75,6.0,3,80.00,15,204,Credit Card,1,32,108,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2023-07-10 13:30:00,30,1,Battle Axe,Combat Tools,95,10.0,5,234.46,0,1172,Credit Card,191,94,702,7,2023
9996,2023-07-10 11:53:00,38,3,Large Axe,Outdoor Equipment,75,6.0,5,158.68,0,793,Mobile Payment,191,63,478,7,2023
9997,2023-07-10 18:31:00,23,7,Battle Axe,Combat Tools,95,10.0,1,234.46,0,234,Mobile Payment,191,94,140,7,2023
9998,2023-07-10 15:30:00,48,6,Battle Axe,Combat Tools,95,10.0,3,234.46,0,703,Cash,191,94,421,7,2023


In [85]:
print( data.groupby( 'product_name' ).size() )

product_name
Battle Axe    3417
Large Axe     3285
Medium Axe    1629
Small Axe     1669
dtype: int64


In [86]:
#Ktory dzien roku
data[ 'transaction_datetime' ] = pd.to_datetime( data[ 'transaction_datetime' ] )
data[ 'day_of_year' ] = data[ 'transaction_datetime' ].dt.dayofyear

In [87]:
print( data )
data.to_csv( 'output.csv', index = False )

     transaction_datetime  customer_id  location product_name  \
0     2018-01-01 17:34:00           67         3    Small Axe   
1     2018-01-01 09:28:00           49         9   Battle Axe   
2     2018-01-01 18:57:00           20         5   Battle Axe   
3     2018-01-01 09:43:00           44         5    Large Axe   
4     2018-01-01 10:01:00           26        13    Large Axe   
...                   ...          ...       ...          ...   
9995  2023-07-10 13:30:00           30         1   Battle Axe   
9996  2023-07-10 11:53:00           38         3    Large Axe   
9997  2023-07-10 18:31:00           23         7   Battle Axe   
9998  2023-07-10 15:30:00           48         6   Battle Axe   
9999  2023-07-10 16:37:00           21         3   Battle Axe   

               category  size_cm  weight_kg  quantity  unit_price  discount  \
0            Hand Tools       35        1.5         5       30.00        15   
1          Combat Tools       95       10.0         2      12

In [88]:
label_encoder = LabelEncoder()
data[ 'location' ] = label_encoder.fit_transform( data[ 'location' ] )
data[ 'product_name' ] = label_encoder.fit_transform( data[ 'product_name' ] )

In [89]:
X = data[ [ 'product_name', 'unit_price', 'location', 'discount', 'day_of_year' ] ]
y = data[ 'quantity' ]

In [90]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2, random_state = 42 )

In [91]:
xg_reg = xgb.XGBRegressor( objective = 'reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 100 )

In [92]:
xg_reg.fit( X_train, y_train )

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [93]:
def predict_profits_in_date_range( data, product_name, new_price, start_date, end_date ):
    data[ 'transaction_month' ] = data[ 'transaction_datetime' ].dt.month
    data[ 'transaction_year' ] = data[ 'transaction_datetime' ].dt.year
    filtered_data = data[ ( data[ 'transaction_datetime' ] >= start_date ) & ( data[ 'transaction_datetime' ] <= end_date ) ]

    historical_profits = ( filtered_data[ 'unit_price' ] - filtered_data[ 'cost_price' ] ) * filtered_data[ 'quantity' ]
    total_historical_profits = historical_profits.sum()
    
    filtered_data_copy = filtered_data.copy()
    filtered_data_copy.loc[ filtered_data_copy[ 'product_name' ] == product_name, 'unit_price' ] = new_price

    new_quantity_predictions = xg_reg.predict( filtered_data_copy[ [ 'product_name', 'unit_price', 'location', 'discount', 'day_of_year' ] ] )
    new_profits = ( filtered_data_copy[ 'unit_price' ] - filtered_data_copy[ 'cost_price' ] ) * new_quantity_predictions

    total_new_profits = new_profits.sum()
    
    return total_historical_profits, total_new_profits

In [97]:
new_price = 22
product_name = 3
discount = 0
start_date = '2023-01-01'
end_date = '2023-06-30'

historical_profits, predicted_profits = predict_profits_in_date_range( data, product_name, new_price, start_date, end_date )

print(f"Historyczne zyski za okres od {start_date} do {end_date}: {historical_profits:.2f}")
print(f"Przewidywane zyski po podwyższeniu ceny: {predicted_profits:.2f}")

Historyczne zyski za okres od 2023-01-01 do 2023-06-30: 241110.42
Przewidywane zyski po podwyższeniu ceny: 226588.03


In [95]:
print( data )

     transaction_datetime  customer_id  location  product_name  \
0     2018-01-01 17:34:00           67         3             3   
1     2018-01-01 09:28:00           49         9             0   
2     2018-01-01 18:57:00           20         5             0   
3     2018-01-01 09:43:00           44         5             1   
4     2018-01-01 10:01:00           26        13             1   
...                   ...          ...       ...           ...   
9995  2023-07-10 13:30:00           30         1             0   
9996  2023-07-10 11:53:00           38         3             1   
9997  2023-07-10 18:31:00           23         7             0   
9998  2023-07-10 15:30:00           48         6             0   
9999  2023-07-10 16:37:00           21         3             0   

               category  size_cm  weight_kg  quantity  unit_price  discount  \
0            Hand Tools       35        1.5         5       30.00        15   
1          Combat Tools       95       10.0      